In [ ]:
import deeplake
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.models import Sequential

ds = deeplake.load("hub://activeloop/vehicles-openimages")
label_map = {
    'Sedan': 'car',
    'Coupe': 'car',
    'Convertible': 'car',
    'Hatchback': 'car',
    'SUV': 'SUV',
    'Pickup': 'truck',
    'Van': 'van',
    'Bus': 'bus',
    'Trailer': 'semi-truck',
    'Tractor-trailer': 'semi-truck',
    'Motorcycle': 'motorcycle',
}


        

: 

In [ ]:
def map_labels(sample):
    new_labels = []
    for label in sample['labels'].np():
        readable_label = label.decode('utf-8') if isinstance(label, bytes) else label
        if readable_label in label_map:
            new_labels.append(label_map[readable_label])
    return new_labels

processed_images = []
processed_labels = []

for sample in ds:
    labels = map_labels(sample)
    if labels:
        img = sample['images'].np()
        img_resized = tf.image.resize(img, (96,96)).np()
        processed_images.append(img_resized)
        processed_labels.append(labels[0])

class_names = sorted(set(label_map.values()))
label_to_index = {name: index for index, name in enumerate(class_names)}
y = [label_to_index[label] for label in processed_labels]
X = np.array(processed_images)/ 255.0


y = to_categorical(y, num_classes=len(class_names))

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(96, 96, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(class_names), activation='softmax')
])